In [449]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [450]:
df = pd.read_csv("../data/data.csv")
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [451]:
y = df["diagnosis"]
y = y.values.reshape(y.shape[0], 1)
X = df[df.columns[2:32]]

train_X, test_X, train_Y, test_Y = train_test_split(X, y, test_size=0.66, random_state=42)

In [452]:
# Standardization
train_X = (train_X - train_X.mean()) / train_X.std()
test_X = (test_X - test_X.mean()) / test_X.std()

In [453]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def predict(w, b, X):
    a = sigmoid(np.dot(X, w) + b)
    return a

def oneORzero(x):
    if x >= 0.5: return 1
    elif x < 0.5: return 0

In [454]:
def random_init(dim):
    return np.zeros((dim, 1)), 0

def propagation(w, b, X, Y):
    m = X.shape[0]

    #forward propagation
    z = np.dot(X, w) + b
    a = sigmoid(z)

    #backward propagation
    dz = a - Y
    dw, db = np.dot(np.transpose(X), dz) / m, np.sum(dz) / m

    return {"dw": dw, "db": db}

def gradient_descent(w, b, X, Y, learning_rate, num_iteration):
    for i in range(num_iteration):
        grads = propagation(w, b, X, Y)
        dw, db = grads["dw"], grads["db"]

        #updating w and b
        w, b = w - learning_rate * dw, b - learning_rate * db

    return {"w": w, "b": b}

In [455]:
w, b = random_init(train_X.shape[1])
params = gradient_descent(w, b, train_X, train_Y, 0.01, 2000)


In [456]:
# Accuracy for training set:
train_prediction = np.array(list(map(oneORzero, predict(params["w"], params["b"], train_X))))
train_prediction = train_prediction.reshape((train_prediction.shape[0], 1))

# Accuracy for test set:
test_prediction = np.array(list(map(oneORzero, predict(params["w"], params["b"], test_X))))
test_prediction = test_prediction.reshape((test_prediction.shape[0], 1))

print("Training set accuracy = ", (100 - np.mean(np.abs(train_prediction - train_Y)) * 100))
print("Test set accuracy = ", (100 - np.mean(np.abs(test_prediction - test_Y)) * 100))

Training set accuracy =  98.44559585492227
Test set accuracy =  97.6063829787234


## SKlearn logistic regression

In [457]:
from sklearn.linear_model import LogisticRegression
lgr = LogisticRegression()
lgr.fit(train_X,train_Y)
print("our accuracy is:{}".format(lgr.score(test_X,test_Y)*100))

our accuracy is:97.6063829787234


c:\users\charl\pycharmprojects\test\venv\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Random Forest

In [458]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
y_pred = lgr.predict(test_X)
classification_report(test_Y, y_pred)

'              precision    recall  f1-score   support\n\n           0       0.98      0.98      0.98       241\n           1       0.97      0.96      0.97       135\n\n    accuracy                           0.98       376\n   macro avg       0.97      0.97      0.97       376\nweighted avg       0.98      0.98      0.98       376\n'